# **TEXT CLASSIFICATION USING BERT**

**APA ITU BERT?**

BERT (Bidirectional Encoder Representations from Transformers) adalah model pre-trained yang dirancang untuk memahami konteks kata dengan membaca teks secara dua arah (kiri dan kanan). BERT unggul dalam tugas seperti text classification karena:

**Transfer Learning**: Sudah dilatih pada dataset besar (Wikipedia, BookCorpus).

**Fine-Tuning**: Dapat disesuaikan untuk tugas spesifik seperti klasifikasi teks.

 **1. EXPLORATORY DATA ANALYSIS AND** **PREPROCESSING**

In [ ]:
! pip install torch torchvision

In [ ]:
! pip install tqdm

In [ ]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
import pandas as pd
#Load dataset
df = pd.read_csv ('/content/data_news_practice.csv')

In [ ]:
df.head(10)

In [ ]:
# Drop satu kolom
df = df.drop(columns=['news_headline'])

Menghapus kolom news_headline, karena kita fokus ke news_article

In [ ]:
df.head(10)

In [ ]:
df.news_category.value_counts()

Menampilkan jumlah data dari setiap label

In [ ]:
possible_labels = df.news_category.unique()

In [ ]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

Merubah news_category menjadi label_dict

In [ ]:
label_dict

Memberi label untuk setiap category, pada case ini kategori technology itu bernilai 0, sports 1, world 2, dst

In [ ]:
df['label'] = df.news_category.replace(label_dict)

merubah label_dict menjadi label

In [ ]:
df.head()

**2. TRAINING AND VALIDATION SPLIT**

In [ ]:
from sklearn.model_selection import train_test_split

train_test_split dari sklearn.model_selection digunakan untuk membagi dataset menjadi dua subset, yaitu subset pelatihan (training set) dan subset pengujian (test set). Proses ini umumnya dilakukan sebagai langkah pertama dalam mengembangkan model machine learning.

In [ ]:
x_train, x_val, y_train, y_val =  train_test_split(df.index.values,
                                                   df.label.values,
                                                   test_size=0.15,
                                                   random_state=17,
                                                   stratify=df.label.values
)

test size 0.15/15% adalah seberapa banyak data ini disiapkan untuk pelatihan

random state adalah parameter yang sering digunakan dalam fungsi-fungsi pembagian data atau model di library Python seperti scikit-learn. Fungsinya adalah mengontrol atau menentukan seed untuk pengacakan (random seed) sehingga proses acak menghasilkan output yang konsisten.


In [ ]:
df['df_type'] = ['not_set']*df.shape[0]

Membuat kolom baru df_type yang akan berisikan not_set untuk semua sample

In [ ]:
df.head()

In [ ]:
df.loc[x_train, 'data_type'] = 'train'
df.loc[x_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['news_category', 'label', 'data_type']).count()

Dari data diatas dapat dilihat bahwa setiap category sudah membagi data train dan data validationnya masing masing

**3. LOADING TOKENIZER AND ENOCODING DATA**

Tokenizer mengambil teks mentah sebagai input dan membaginya menjadi Token, angka
numerik yang mewakili kata tertentu. Tokenizer mengubah teks menjadi data numerik.

In [ ]:
! pip install transformers

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

**TOKENIZER**

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

Tokenizer diambil dari BERT yang sudah di latih sebelumnya

bert-base-uncased' berarti harus menggunakan semua data huruf kecil

do_lower_case digunakan untuk mengubah semua data menjadi huruf kecil.

**ENCODING**

Selanjutnya pada tahap ini kita akan mengkonversikan semua data ke dalam bentuk yang
disandikan.

In [ ]:
# Encoding the Training data
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].news_article.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=512,
    return_tensors='pt'
)

# Encoding the Validation data
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].news_article.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=512,
    return_tensors='pt'
)

# Spliting the data for the BERT training
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

max_length untuk BERT LM itu biasanya antara 128/256/512 (bisa lebih jika model yang digunakan berbeda), max_length itu untuk menghitung jumlah kata pada setiap kolom data.

pada case ini, max length yang digunakan 512 karena data yang di analisis itu adalah artikel dari berita

**4. CHANGE INPUT INTO BERT ALGORITHM**

In [ ]:
# Creating two different dataset
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train)

1364 data akan menjadi training

In [ ]:
len(dataset_val)

241 data akan di validasi

**5. SETTING UP BERT PRE TRAINED MODEL**

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Memuat sebuah model kecerdasan buatan yang bernama BERT, yang sudah dilatih untuk mengenali jenis-jenis kategori teks tertentu

**6. CREATING DATA LOADER**

Dataloader menggabungkan kumpulan data dan sampler, dan menyediakan iterator tunggal
atau multiproses di atas kumpulan data. Dataloader yang akan digunakan ada 2, yaitu
dataloader untuk data train dan validation

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 16

# We Need two different dataloder
dataloader_train = DataLoader(dataset_train,
                              sampler=RandomSampler(dataset_train),
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val,
                              sampler=RandomSampler(dataset_val),
                              batch_size=batch_size)

**7. SETTING UP OPTIMISER AND SCHEDULER**

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5,
                  eps=1e-8)

Algoritma AdamW: menghitung peluruhan bobot sebelum menerapkan langkah gradien.

In [ ]:
epochs = 3

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

Epoch menunjukkan berapa kali kita ingin melatih data kita, disini kita akan melakukan pelatihan sebanyak 3x

**7. DEFINING OUR PERFORMANCE METRICS**

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def f1_score_func(preds, labels):

    # Setting up the preds to axis=1
    # Flatting it to a single iterable list of array
    preds_flat = np.argmax(preds, axis=1).flatten()

    # Flattening the labels
    labels_flat = labels.flatten()

    # Returning the f1_score as define by sklearn
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    # Iterating over all the unique labels
    # label_flat are the --> True labels
    for label in np.unique(labels_flat):
        # Taking out all the pred_flat where the True alable is the lable we care about.
        # e.g. for the label Happy -- we Takes all Prediction for true happy flag
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

**8. IMPORT TRAINING LOOP TO CONTROL PYTORCH FINETUNING OF BERT USING CPU OR GPU ACCELERATION**

Pada tahap ini membuat loop pelatihan untuk mengontrol finetuning PyTorch BERT
menggunakan akselerasi CPU atau GPU.

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

In [ ]:
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in tqdm(dataloader_val):

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

Setiap epoch akan melakukan training dan validation dan akan menghasilkan training loss,
validation loss, dan F1-Score.

• Training loss adalah kesalahan pada training set data

• Validation loss adalah kesalahan setelah menjalankan set validasi data melalui
jaringan yang dilatih.

• F1-Score adalah rata rata dari precision dan recall

In [ ]:
for epoch in tqdm(range(1, epochs+1)):

    model.train()

    loss_train_total = 0

    # Setting up the Progress bar to Moniter the progress of training
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad() # As we not working with thew RNN's

        # As our dataloader has '3' iteams so batches will be the Tuple of '3'
        batch = tuple(b.to(device) for b in batch)

        # INPUTS
        # Pulling out the inputs in the form of dictionary
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        # OUTPUTS
        outputs = model(**inputs) # '**' Unpacking the dictionary stright into the input

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()           # backpropagation

        # Gradient Clipping -- Taking the Grad. & gives it a NORM value ~ 1
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')

    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

• Jika training loss dan validation loss terus turun, naikan epoch dua kali lipatnya. Katakanlah epoch selanjutnya 6.

• Jika pada epoch 6 training loss dan validation loss terlihat stagnan (tidak mengalami
penurunan), coba naikan kompleksitas mode (menambah hidden layer).

• Jika pada epoch 6, training loss dan validation loss berlawanan (validation loss
meningkat dan training loss menurun, atau sebaliknya), maka itu mengindikasikan
model kita overfitting. Overfitting adalah suatu keadaan dimana data yang digunakan
untuk pelatihan itu adalah yang "terbaik". Sehingga apabila dilakukan tes dengan
menggunakan data yang berbeda dapat mengurangi akurasi (hasil yang dibuat tidak
sesuai yang diharapkan).

• Jika pada epoch 6, validation loss jauh lebih kecil dibandingkan training loss ini
mengindikasikan underfitting. Underfitting adalah keadaan dimana model pelatihan
data yang dibuat tidak mewakilkan keseluruhan data yang akan digunakan nantinya.
Sehingga menghasilkan performa yang buruk dalam pelatihan data.

• Dikarenakan output tersebut menghasilkan training loss dan validation loss yang terus
menurun, maka perlu menaikkan epoch dua kali lipatnya untuk percobaan selanjutnya.
Akan tetapi pada F1-Score sudah mendapatkan angka yang optimal yaitu 0.85, maka model ini yang akan kita gunakan untuk testing

**9. LOADING FINETUNED BERT MODEL AND EVALUATE ITS PERFORMANCE**

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

**LOADING BEST BERT MODEL**

In [ ]:
model.load_state_dict(torch.load('/content/finetuned_BERT_epoch_3.model', map_location=torch.device('cpu')))

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

dari model epoch 3, terdapat hasil bahwa model ini memiliki akurais yang tinggi dalam memprediksi kelas kelasnya, seperti pada class technology, dapat dilihat bahwa model ini bekerja dengan nilai akurasi 27 dari 37, sehingga dapat dikatakan model ini memiliki akurasi yang tinggi